In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [ ]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
PATH = 'data/dogbreeds/'

In [ ]:
sz=220

### Load Data

In [ ]:
df = pd.read_csv(PATH+'labels.csv')

In [ ]:
df.columns

In [ ]:
df['id'].size

In [ ]:
test_file = PATH+'train/'+df['id'][0]+'.jpg'
print(test_file)
test_img = plt.imread(test_file)
plt.imshow(test_img)
# PIL.Image.open(test_file)

In [ ]:
size_map = { k:plt.imread(f'{PATH}train/{k}.jpg').shape for k in df['id'][1:]}

In [ ]:
size_map

In [ ]:
h, w, ch = list(zip(*size_map.values()))

In [ ]:
plt.hist(w)

In [ ]:
w = np.array(w)
plt.hist(w[w<600])

### Architecture

In [ ]:
!rm -rf {PATH}tmp

In [ ]:
arch=resnet34

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
n = df['id'].size-1
val_idxs = get_cv_idxs(n)
data = ImageClassifierData.from_csv(path=PATH, folder='train', csv_fname=f'{PATH}labels.csv', test_name='test', tfms=tfms, suffix='.jpg', val_idxs=val_idxs)
new_data = data.resize(sz, 'tmp')

In [ ]:
learn = ConvLearner.pretrained(arch, data=new_data, precompute=True)

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(lrs=1e-1, n_cycle=2)

In [ ]:
learn.fit(lrs=1e-2, n_cycle=2)

In [ ]:
learn.precompute=False

In [ ]:
learn.fit(lrs=1e-2, n_cycle=3)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(lrs=[1e-4,1e-3,1e-2], n_cycle=3, cycle_len=1)

In [ ]:
learn.fit(lrs=[1e-4,1e-3,1e-2], n_cycle=3, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds,y), metrics.log_loss(y, probs)

In [ ]:
preds = np.argmax(log_preds, axis=1)
# np.array(preds==y).mean()
preds
# accuracy(log_preds,y)